In [1]:
import numpy as np
import pandas as pd

This notebook is a follow through the Schaake shuffle paper 
- https://journals.ametsoc.org/view/journals/hydr/5/1/1525-7541_2004_005_0243_tssamf_2_0_co_2.xml

It shows that the shuffle is equivalent to quantile mapping.

-------

Use the data example from the paper:

<img src="variables.png" alt="drawing" width="400"/>

In [2]:
x = [15.3, 11.2, 8.8, 11.9, 7.5, 9.7, 8.3, 12.5, 10.3, 10.1]
y = [10.7, 9.3, 6.8, 11.3, 12.2, 13.6, 8.9, 9.9, 11.8, 12.9]

In [3]:
X = np.sort(x) # chi
Y = np.sort(y) # gamma
B = np.array(sorted(np.arange(len(y)), key=lambda i: y[i]))

print(f"{X = }")
print(f"{Y = }")
print(f"{B = } - note python is zero indexed")

X = array([ 7.5,  8.3,  8.8,  9.7, 10.1, 10.3, 11.2, 11.9, 12.5, 15.3])
Y = array([ 6.8,  8.9,  9.3,  9.9, 10.7, 11.3, 11.8, 12.2, 12.9, 13.6])
B = array([2, 6, 1, 7, 0, 3, 8, 4, 9, 5]) - note python is zero indexed


The algorithm for the shuffle proceedure is:

<img src="xss_method.png" alt="drawing" width="400"/>

In [25]:
# This is the implementation

Xss = [0]*len(y)
for r in range(len(y)):
    Xss[B[r]]=X[r]

This should give the result from the paper:

<img src="xss_results.png" alt="drawing" width="400"/>

In [26]:
# which it does
print(f"{Xss = }")

Xss = [10.1, 8.8, 7.5, 10.3, 11.9, 15.3, 8.3, 9.7, 11.2, 12.5]


In [27]:
# Rewrite the shuffle as a function

def schaake_shuffle(x, y):
    X = np.sort(x)
    B = sorted(np.arange(len(y)), key=lambda i: y[i])
    Xss = [0]*len(y)
    for r in range(len(y)):
        Xss[B[r]]=X[r]
    return Xss

In [7]:
# The shuffle is essentially quantile mapping. 
# We make this even clearer in version of function below

def rank_map(x, y):
    """
    
    This version is easier to see its connection to quantile mapping.
    This is quantile mapping y with respect to x.
    
    """
    X = np.sort(x)
    Y = np.sort(y)
    yx = []
    for yi in y:
        # if yi is nth largest in y select nth largest in x
        yx += [X[np.argwhere(Y==yi).item()]]
    return yx

Sanity check - The two functions give the same results on the original data

In [8]:
schaake_shuffle(x, y)

[10.1, 8.8, 7.5, 10.3, 11.9, 15.3, 8.3, 9.7, 11.2, 12.5]

In [9]:
rank_map(x, y)

[10.1, 8.8, 7.5, 10.3, 11.9, 15.3, 8.3, 9.7, 11.2, 12.5]

Lets use the full table from the paper

<img src="x_table.png" alt="drawing" width="400"/>


In [24]:
x1 = [7.5, 8.3, 8.8, 9.7, 10.1, 10.3, 11.2, 11.9, 12.5, 15.3]
x2 = [6.3, 7.2, 7.5, 7.9, 8.6, 9.3, 11.8, 12.2, 13.5, 17.7]
x3 = [12.4, 13.5, 14.2, 14.5, 15.6, 15.9, 16.3, 17.6, 18.3, 23.9]

pd.DataFrame({'Stn 1':x1, 'Stn 2':x2, 'Stn 3':x3})

,Stn 1,Stn 2,Stn 3
0,7.5,6.3,12.4
1,8.3,7.2,13.5
2,8.8,7.5,14.2
3,9.7,7.9,14.5
4,10.1,8.6,15.6
5,10.3,9.3,15.9
6,11.2,11.8,16.3
7,11.9,12.2,17.6
8,12.5,13.5,18.3
9,15.3,17.7,23.9


<img src="y_table.png" alt="drawing" width="400"/>

In [28]:
y1 = [10.7, 9.3, 6.8, 11.3, 12.2, 13.6, 8.9, 9.9, 11.8, 12.9]
y2 = [10.9, 9.1, 7.2, 10.7, 13.1, 14.2, 9.4, 9.2, 11.9, 12.5]
y3 = [13.5, 13.7, 9.3, 15.6, 17.8, 19.3, 12.1, 11.8, 15.2, 16.9]

pd.DataFrame({'Stn 1':y1, 'Stn 2':y2, 'Stn 3':y3})

,Stn 1,Stn 2,Stn 3
0,10.7,10.9,13.5
1,9.3,9.1,13.7
2,6.8,7.2,9.3
3,11.3,10.7,15.6
4,12.2,13.1,17.8
5,13.6,14.2,19.3
6,8.9,9.4,12.1
7,9.9,9.2,11.8
8,11.8,11.9,15.2
9,12.9,12.5,16.9


Th expected results of running the shuffle on this data is:

<img src="xss_table.png" alt="drawing" width="400"/>

In [29]:
# The shuffle function we wrote recreates it
pd.DataFrame({
    f"Stn 1": schaake_shuffle(x1,y1),
    f"Stn 2": schaake_shuffle(x2,y2),
    f"Stn 3": schaake_shuffle(x3,y3),
})

,Stn 1,Stn 2,Stn 3
0,10.1,9.3,14.5
1,8.8,7.2,15.6
2,7.5,6.3,12.4
3,10.3,8.6,16.3
4,11.9,13.5,18.3
5,15.3,17.7,23.9
6,8.3,7.9,14.2
7,9.7,7.5,13.5
8,11.2,11.8,15.9
9,12.5,12.2,17.6


In [30]:
# The quantile mapping function we wrote also recreates it
pd.DataFrame({
    f"Stn 1": rank_map(x1,y1),
    f"Stn 2": rank_map(x2,y2),
    f"Stn 3": rank_map(x3,y3),
})

,Stn 1,Stn 2,Stn 3
0,10.1,9.3,14.5
1,8.8,7.2,15.6
2,7.5,6.3,12.4
3,10.3,8.6,16.3
4,11.9,13.5,18.3
5,15.3,17.7,23.9
6,8.3,7.9,14.2
7,9.7,7.5,13.5
8,11.2,11.8,15.9
9,12.5,12.2,17.6


So the Schaake shuffle is simply an empirical version of quantile mapping where we map the observations to the 1d distributions of the model data